In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import division, print_function
import os

import warnings

from keras import backend as K
from keras.layers import Input
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras.layers import Conv2D
from keras.regularizers import l2
from keras.layers.core import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.utils import layer_utils


WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16_Hybrid_1365(include_top=True, weights='places',
                      input_tensor=None, input_shape=None,
                      pooling=None,
                      classes=1365):
    """InstantiatesInstantiates the VGG-ImageNetPlaces365-hybrid architecture.
    Optionally loads weights pre-trained
    on Places. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
                 'places' (pre-training on Places),
                 or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`, or invalid input shape
        """

    if not (weights in {'places', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `places` '
                         '(pre-training on Places), '
                         'or the path to the weights file to be loaded.')

    if weights == 'places' and include_top and classes != 1365:
        raise ValueError('If using `weights` as places with `include_top`'
                         ' as true, `classes` should be 1365')


    # Determine proper input shape
    input_shape = (224,224,3)


    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor



    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block1_conv1')(img_input)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)



    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)



    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)



    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)



    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
                     kernel_regularizer=l2(0.0002),
                     activation='relu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)



    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dropout(0.5, name='drop_fc1')(x)

        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dropout(0.5, name='drop_fc2')(x)

        x = Dense(1365, activation='softmax', name='predictions')(x)

    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
       
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input


    # Create model.
    model = Model(inputs,x, name='vgg16-hybrid1365')

    # load weights
    if weights == 'places':
        if include_top:
            weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')

        model.load_weights(weights_path)

        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')


    elif weights is not None:
        model.load_weights(weights)



    return model

In [0]:
def create_features_exctractor(model, layer_name='fc1'):
    extractor = Model(inputs=model.input,
                      outputs=model.get_layer(layer_name).output)
    return extractor

In [0]:
from keras.applications.vgg16 import VGG16
#m= VGG16(weights='imagenet', include_top=True)
m = VGG16_Hybrid_1365(include_top=True)
network = create_features_exctractor(m)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
559480832/559473444 [==============================] - 6s 0us/step


In [0]:
#network2 = VGG16_Hybrid_1365(include_top=False,pooling='avg')
network.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
!pip uninstall -y kaggle 
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME =team13eao2
%env KAGGLE_KEY =dfee73d25376e360d2ea7e0de57e9d1b
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=6504bd67f9c12cf4bc84dd6c14c686a92cc413460ac4426007196ec9065e0cf1
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
env: KAGGLE_USERNAME=team13eao2
env: KAGGLE_KEY=dfee73d25376e360d2ea7e0de57e9d1b
 99% 716M/724M [00:05<00:00, 170MB/s]
100% 724M/724M [00:05<00:00, 147MB/s]
Archive:  /content/fcis-sc-deeplearning-competition.zip
  inflating: submit.csv              
  inflating: test/test/003_1b.jpg    
  inflating: test/test/0310_piscines_8.jpg  
  inflating: test/test/051011Greenhouse.jpg  
  inflating: test/test/100_1238.jpg  
  inflating: test/test/1164876979_0.jpg  
  inflating: test/test/1208_hall_entrada_010.jpg  
  inflating: test/test/12_t1_H_S_Pool_Inside.jpg  
  inflating: test/test/13_2o_anda

In [0]:
import numpy as np 
import pandas as pd 
import cv2
import os 
path = "/content/train/train"
dicto = dict()
dicto['airport_inside']=1
dicto['bakery']=2
dicto['bedroom']=3 
dicto['greenhouse']=4
dicto['gym']=5
dicto['kitchen']=6
dicto['operating_room']=7
dicto['poolinside']=8
dicto['restaurant']=9
dicto['toystore']=10
data = []
ctr = 0 
for folder in os.listdir(path):
    print(folder)
    for item in os.listdir(path+"/"+folder):
       ctr+=1
       tmp=path+"/"+folder+"/"+item
       data.append(tuple((tmp,dicto[folder])))
    print(ctr)
    ctr=0
data = np.array(data)
X=data[:,0]
Y=data[:,1]

greenhouse
81
gym
185
operating_room
115
poolinside
140
bedroom
530
toystore
277
airport_inside
487
bakery
324
kitchen
588
restaurant
411


In [0]:
 from sklearn.model_selection import train_test_split
 np.random.seed(42)
 X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify=Y, test_size=0.2, random_state=42)


In [0]:
np.shape(X_test)

(628,)

In [0]:
ls = [] 
for item in y_train: 
    ls.append(int(item))
y_train=np.array(ls)

In [0]:
ls = [] 
for item in y_test: 
    ls.append(int(item))
y_test=np.array(ls)

In [0]:
ctr = np.zeros(11)
for i in range(np.shape(y_train)[0]):
    ctr[y_train[i]]+=1
print(ctr)
ctr = np.array([0,111,241,76,435,352,30,408,389,171,278])

[  0. 389. 259. 424.  65. 148. 470.  92. 112. 329. 222.]


In [0]:
def load_data(paths):
    i = 0
    images=[]
    for path in paths: 
        i+=1
        img = cv2.imread(path)
        if img is None : 
          print("aheh")
          continue
        img = cv2.resize(img,(224,224))
        images.append(img)
        print(i)
    images=np.array(images)
    return images

In [0]:
X_train = load_data(X_train)
X_test = load_data(X_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [0]:
np.shape(y_test)

(628,)

In [0]:
X_aug = [] 
Y_aug = [] 
for i in range(np.shape(X_train)[0]):
    it = train_datagen.flow(np.reshape(X_train[i],(1,224,224,3)),batch_size=1)
    batch = it.next()
    image = np.reshape(batch[0],(224,224,3))
    X_aug.append(image)
    Y_aug.append(y_train[i])
    if ctr[y_train[i]]!=0 and (y_train[i]==1 or y_train[i]==2 or y_train[i]==3 or y_train[i]==6 or y_train[i]==9) : 
       ctr[y_train[i]]-=1
       batch = it.next()
       image = np.reshape(batch[0],(224,224,3))
       X_aug.append(image)
       Y_aug.append(y_train[i])
    elif ctr[y_train[i]]!=0 and y_train[i]==10:
        
         for k in range(2):
           ctr[y_train[i]]-=1
           batch = it.next()
           image = np.reshape(batch[0],(224,224,3))
           X_aug.append(image)
           Y_aug.append(y_train[i])
           if ctr[y_train[i]]==0:
              break
    elif ctr[y_train[i]]!=0 and y_train[i]==5:
         
         for k in range(3):
           ctr[y_train[i]]-=1
           batch = it.next()
           image = np.reshape(batch[0],(224,224,3))
           X_aug.append(image)
           Y_aug.append(y_train[i])
           if ctr[y_train[i]]==0:
              break
    elif ctr[y_train[i]]!=0 and y_train[i]==8:
         
         for k in range(4):
           ctr[y_train[i]]-=1
           batch = it.next()
           image = np.reshape(batch[0],(224,224,3))
           X_aug.append(image)
           Y_aug.append(y_train[i])
           if ctr[y_train[i]]==0:
              break
    elif ctr[y_train[i]]!=0 and y_train[i]==7:
         
         for k in range(5):
           ctr[y_train[i]]-=1
           batch = it.next()
           image = np.reshape(batch[0],(224,224,3))
           X_aug.append(image)
           Y_aug.append(y_train[i])
           if ctr[y_train[i]]==0:
              break
    elif ctr[y_train[i]]!=0 and y_train[i]==4:
         
         for k in range(7):
           ctr[y_train[i]]-=1
           batch = it.next()
           image = np.reshape(batch[0],(224,224,3))
           X_aug.append(image)
           Y_aug.append(y_train[i])
           if ctr[y_train[i]]==0:
              break
    print(i)
print(np.shape(X_aug))
print(np.shape(Y_aug))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
#X_aug = X_train
#X_test = X_test
#np.shape(X_test)
#network = load_model("/content/drive/My Drive/VGG16_4096.h5")

(627, 224, 224, 3)

In [0]:
features = network.predict(np.array(X_aug))
features_test= network.predict(X_test)

In [0]:
X_aug = np.array(X_aug)
#Y_aug = np.array(y_train)
lsa = [] 
for i in range(np.shape(y_test)[0]):
    if i ==613:
       continue
    else:
      lsa.append(y_test[i])
print(np.shape(lsa))
#features = network.predict(X_aug)

(627,)


In [0]:
np.shape(X_test)
y_test = np.array(lsa)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [0]:
def Metric(label,prediction):
    total = np.zeros(11)
    true  = np.zeros(11)
    for i in range(np.shape(label)[0]):
        total[label[i]]+=1
        if prediction[i]==label[i]: 
           true[prediction[i]]+=1
    for i in range(11):
        if i==0:
           continue
        print("Class"+str(i)+" : "+str(true[i])+" / "+str(total[i]))

In [0]:
from keras.utils import to_categorical
y_train_cat = np.array(Y_aug) -1 
y_train_cat = to_categorical(y_train_cat,10)
y_test_cat = y_test-1 
y_test_cat = to_categorical(y_test_cat,10)

In [0]:
model1= Sequential()
model1.add(Dense(2048, input_dim=4096,activation='relu'))
model1.add(Dense(1024,activation='relu'))
model1.add(Dense(512,activation='relu'))
model1.add(Dense(128,activation='relu'))
model1.add(Dense(64,activation='relu'))
model1.add(Dense(10,activation='softmax'))
model1.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
np.shape(y_train_cat)

(5001, 10)

In [0]:
model1.fit(features,y_train_cat,validation_data=(features_test,y_test_cat),epochs=25)

Train on 5001 samples, validate on 627 samples
Epoch 1/25
5001/5001 [==============================] - 3s 538us/step - loss: 0.8526 - acc: 0.7574 - val_loss: 0.2638 - val_acc: 0.9234
Epoch 2/25
5001/5001 [==============================] - 1s 267us/step - loss: 0.2551 - acc: 0.9170 - val_loss: 0.3755 - val_acc: 0.9219
Epoch 3/25
5001/5001 [==============================] - 1s 270us/step - loss: 0.1281 - acc: 0.9588 - val_loss: 0.3054 - val_acc: 0.9394
Epoch 4/25
5001/5001 [==============================] - 1s 265us/step - loss: 0.1026 - acc: 0.9680 - val_loss: 0.5586 - val_acc: 0.9075
Epoch 5/25
5001/5001 [==============================] - 1s 263us/step - loss: 0.0715 - acc: 0.9790 - val_loss: 0.4001 - val_acc: 0.9266
Epoch 6/25
5001/5001 [==============================] - 1s 265us/step - loss: 0.0720 - acc: 0.9814 - val_loss: 0.5117 - val_acc: 0.9091
Epoch 7/25
5001/5001 [==============================] - 1s 264us/step - loss: 0.0676 - acc: 0.9816 - val_loss: 0.5136 - val_acc: 0.9250
E

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
model2 = Sequential()
model2.add(Dense(1024, input_dim=4096,activation='relu'))
model2.add(Dense(512,activation='relu'))
model2.add(Dense(256,activation='relu'))
model2.add(Dense(128,activation='relu'))
model2.add(Dense(64,activation='relu'))
model2.add(Dense(10,activation='softmax'))
model2.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model2.fit(features,y_train_cat,validation_data=(features_test,y_test_cat),epochs=25)

Train on 5001 samples, validate on 627 samples
Epoch 1/25
5001/5001 [==============================] - 2s 465us/step - loss: 0.7321 - acc: 0.7724 - val_loss: 0.3243 - val_acc: 0.9139
Epoch 2/25
5001/5001 [==============================] - 1s 218us/step - loss: 0.2575 - acc: 0.9174 - val_loss: 0.4078 - val_acc: 0.9075
Epoch 3/25
5001/5001 [==============================] - 1s 220us/step - loss: 0.1755 - acc: 0.9434 - val_loss: 0.3616 - val_acc: 0.9187
Epoch 4/25
5001/5001 [==============================] - 1s 217us/step - loss: 0.0792 - acc: 0.9746 - val_loss: 0.4278 - val_acc: 0.9043
Epoch 5/25
5001/5001 [==============================] - 1s 221us/step - loss: 0.0583 - acc: 0.9804 - val_loss: 0.3768 - val_acc: 0.9346
Epoch 6/25
5001/5001 [==============================] - 1s 219us/step - loss: 0.0757 - acc: 0.9764 - val_loss: 0.5842 - val_acc: 0.9059
Epoch 7/25
5001/5001 [==============================] - 1s 210us/step - loss: 0.0665 - acc: 0.9808 - val_loss: 0.6996 - val_acc: 0.9011
E

In [0]:
model3 = Sequential()
model3.add(Dense(512, input_dim=4096,activation='relu'))
model3.add(Dense(256,activation='relu'))
model3.add(Dense(128,activation='relu'))
model3.add(Dense(64,activation='relu'))
model3.add(Dense(32,activation='relu'))
model3.add(Dense(10,activation='softmax'))
model3.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model3.fit(features,y_train_cat,validation_data=(features_test,y_test_cat),epochs=25)

Train on 5001 samples, validate on 627 samples
Epoch 1/25
5001/5001 [==============================] - 2s 467us/step - loss: 0.7695 - acc: 0.7566 - val_loss: 0.3098 - val_acc: 0.9123
Epoch 2/25
5001/5001 [==============================] - 1s 201us/step - loss: 0.2504 - acc: 0.9194 - val_loss: 0.4239 - val_acc: 0.8931
Epoch 3/25
5001/5001 [==============================] - 1s 194us/step - loss: 0.1361 - acc: 0.9580 - val_loss: 0.3520 - val_acc: 0.9155
Epoch 4/25
5001/5001 [==============================] - 1s 193us/step - loss: 0.0755 - acc: 0.9770 - val_loss: 0.4789 - val_acc: 0.9091
Epoch 5/25
5001/5001 [==============================] - 1s 207us/step - loss: 0.0864 - acc: 0.9738 - val_loss: 0.4631 - val_acc: 0.9091
Epoch 6/25
5001/5001 [==============================] - 1s 192us/step - loss: 0.0694 - acc: 0.9788 - val_loss: 0.4711 - val_acc: 0.9203
Epoch 7/25
5001/5001 [==============================] - 1s 196us/step - loss: 0.0756 - acc: 0.9750 - val_loss: 0.4850 - val_acc: 0.9203
E

In [0]:
prd = model3.predict(features_test)
prd=np.argmax(prd,axis=1)
prd+=1
Metric(y_test,prd)
accuracy_score(y_test,prd)

Class1 : 94.0 / 98.0
Class2 : 59.0 / 65.0
Class3 : 102.0 / 106.0
Class4 : 16.0 / 16.0
Class5 : 31.0 / 37.0
Class6 : 109.0 / 118.0
Class7 : 21.0 / 23.0
Class8 : 26.0 / 27.0
Class9 : 75.0 / 82.0
Class10 : 43.0 / 55.0


0.9186602870813397

In [0]:
from sklearn.metrics import accuracy_score
prds = []
q1 = model1.predict(features_test)
q2 = model2.predict(features_test)
q3 = model3.predict(features_test)
#q4 = model4.predict(features2_test)
tmp = q1+q2+q3
tmp/=3
tmp = np.argmax(tmp,axis=1)
tmp+=1
Metric(y_test,tmp)
accuracy_score(y_test,tmp)


Class1 : 90.0 / 98.0
Class2 : 56.0 / 65.0
Class3 : 103.0 / 106.0
Class4 : 16.0 / 16.0
Class5 : 36.0 / 37.0
Class6 : 113.0 / 118.0
Class7 : 21.0 / 23.0
Class8 : 27.0 / 27.0
Class9 : 74.0 / 82.0
Class10 : 46.0 / 55.0


0.9282296650717703

In [0]:
X = [] 
Y = [] 
#X2=[]
#Y2=[]
#for item in features2: 
 #   X2.append(item)
#for item in features2_test:
 #   X2.append(item)
for item in features: 
   X.append(item)
for item in features_test : 
   X.append(item)
for item in y_train_cat: 
  Y.append(item)
for item in y_test_cat: 
  Y.append(item)

In [0]:
X = np.array(X)
Y = np.array(Y)
#Y2 = np.array(Y)

In [0]:
from keras.callbacks import ModelCheckpoint
filepath = "/content/drive/My Drive/sub_22_Model1_2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model1.fit(X,Y,epochs=30,callbacks=callbacks_list)

Epoch 1/30
5628/5628 [==============================] - 1s 261us/step - loss: 0.1481 - acc: 0.9749

Epoch 00001: loss improved from inf to 0.14810, saving model to /content/drive/My Drive/sub_22_Model1_2.h5
Epoch 2/30
5628/5628 [==============================] - 1s 262us/step - loss: 0.0719 - acc: 0.9863

Epoch 00002: loss improved from 0.14810 to 0.07188, saving model to /content/drive/My Drive/sub_22_Model1_2.h5
Epoch 3/30
5628/5628 [==============================] - 1s 264us/step - loss: 0.0753 - acc: 0.9892

Epoch 00003: loss did not improve from 0.07188
Epoch 4/30
5628/5628 [==============================] - 1s 260us/step - loss: 0.0504 - acc: 0.9909

Epoch 00004: loss improved from 0.07188 to 0.05044, saving model to /content/drive/My Drive/sub_22_Model1_2.h5
Epoch 5/30
5628/5628 [==============================] - 2s 267us/step - loss: 0.0201 - acc: 0.9970

Epoch 00005: loss improved from 0.05044 to 0.02009, saving model to /content/drive/My Drive/sub_22_Model1_2.h5
Epoch 6/30
56

In [0]:
#from keras.callbacks import ModelCheckpoint
# define the checkpoint
#filepath = "/content/drive/My Drive/sub_12_Model1.h5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]
#model1.fit(X,Y,epochs=15,callbacks=callbacks_list)

In [0]:
filepath = "/content/drive/My Drive/sub_22_Model2_2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model2.fit(X,Y,epochs=30,callbacks=callbacks_list)

Epoch 1/30
5628/5628 [==============================] - 1s 218us/step - loss: 0.1128 - acc: 0.9812

Epoch 00001: loss improved from inf to 0.11283, saving model to /content/drive/My Drive/sub_22_Model2_2.h5
Epoch 2/30
5628/5628 [==============================] - 1s 213us/step - loss: 0.0952 - acc: 0.9861

Epoch 00002: loss improved from 0.11283 to 0.09517, saving model to /content/drive/My Drive/sub_22_Model2_2.h5
Epoch 3/30
5628/5628 [==============================] - 1s 213us/step - loss: 0.0458 - acc: 0.9908

Epoch 00003: loss improved from 0.09517 to 0.04581, saving model to /content/drive/My Drive/sub_22_Model2_2.h5
Epoch 4/30
5628/5628 [==============================] - 1s 210us/step - loss: 0.0314 - acc: 0.9931

Epoch 00004: loss improved from 0.04581 to 0.03142, saving model to /content/drive/My Drive/sub_22_Model2_2.h5
Epoch 5/30
5628/5628 [==============================] - 1s 212us/step - loss: 0.0143 - acc: 0.9966

Epoch 00005: loss improved from 0.03142 to 0.01434, saving m

In [0]:
filepath = "/content/drive/My Drive/sub_22_Model3_2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model3.fit(X,Y,epochs=30,callbacks=callbacks_list)

Epoch 1/30
5628/5628 [==============================] - 1s 199us/step - loss: 0.0891 - acc: 0.9856

Epoch 00001: loss improved from inf to 0.08910, saving model to /content/drive/My Drive/sub_22_Model3_2.h5
Epoch 2/30
5628/5628 [==============================] - 1s 186us/step - loss: 0.0496 - acc: 0.9885

Epoch 00002: loss improved from 0.08910 to 0.04963, saving model to /content/drive/My Drive/sub_22_Model3_2.h5
Epoch 3/30
5628/5628 [==============================] - 1s 188us/step - loss: 0.0360 - acc: 0.9918

Epoch 00003: loss improved from 0.04963 to 0.03597, saving model to /content/drive/My Drive/sub_22_Model3_2.h5
Epoch 4/30
5628/5628 [==============================] - 1s 186us/step - loss: 0.0417 - acc: 0.9922

Epoch 00004: loss did not improve from 0.03597
Epoch 5/30
5628/5628 [==============================] - 1s 209us/step - loss: 0.0069 - acc: 0.9986

Epoch 00005: loss improved from 0.03597 to 0.00687, saving model to /content/drive/My Drive/sub_22_Model3_2.h5
Epoch 6/30
56

In [0]:
path = '/content/test/test'
test = []
i = 0 
for item in os.listdir(path):
    if item !="test":
       i+=1
       test.append(item)
       
print(i)
print(test)

772
['airport_inside_0589.jpg', 'airport_inside_0078.jpg', 'restaurant_22_08_altavista.jpg', 'imagem_003.jpg', '0310_piscines_8.jpg', 'indoor015.jpg', 'boulangerie_28_03_yahoo.jpg', 'Pool_inside2_1.jpg', 'pool_inside.jpg', 'int734.jpg', 'int273.jpg', 'airport_inside_0352.jpg', 'airport_inside_0380.jpg', 'at_00_05_8293_15_l.jpg', 'int464.jpg', 'gimnasio_10_08_altavista.jpg', 'airport_inside_0240.jpg', 'dsc04544.jpg', 'airport_inside_0429.jpg', 'int868.jpg', 'int601.jpg', 'gym_14_10_altavista.jpg', 'airport_inside_0348.jpg', 'airport_inside_0581.jpg', 'roomscan40.jpg', 'restaurant_40_02_altavista.jpg', 'operating_room_04_19_altavista.jpg', 'SoftffL.T.jpg', 'speelgoed_58_24_flickr.jpg', 'operating_room_08_09_altavista.jpg', 'boulangerie_51_03_yahoo.jpg', 'airport_inside_0031.jpg', 'int330.jpg', 'restaurant_47_08_altavista.jpg', 'kitchen058.jpg', 'room332.jpg', 'thumb_992881_RickenbackersBistroRestaurant_.jpg', 'indoor_0097.jpg', 'img_0103.jpg', 'b6.jpg', 'restaurant_37_15_altavista.jpg', 

In [0]:
path = '/content/test/test/'
ctrs = []
ans =[]
i=0
for item in test : 
    img = cv2.imread(path+item)
    if img is None : 
       print(item)
       ctrs.append(item)
       ans.append(4)
       continue
    else:
       ctrs.append(item)
       img=cv2.resize(img,(224,224))
       zec=np.reshape(img,(1,224,224,3))
       z = network.predict(zec)
       e1= model1.predict(z)
       e2 = model2.predict(z)
       e3 = model3.predict(z)
       #z2 = mod.predict(zec)
       #e4 = model4.predict(z2)
       #e5 = r.predict(z)
       e  = e1+e2+e3
       #+e4
       e/=3
       e = np.argmax(e)+1
       ans.append(int(e))
    i+=1
    print(i)

     
     

print(np.shape(ctrs))
print(np.shape(ans))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
q = []
for item in ans : 
    print(item)
    if item == 1 : 
       q.append(1)
    elif item==2: 
       q.append(2)
    elif item==3: 
       q.append(3)
    elif item==4: 
       q.append(4)
    elif item==5: 
       q.append(5)
    elif item==6: 
       q.append(6)
    elif item==7: 
       q.append(7)
    elif item==8: 
       q.append(8)
    elif item==9: 
       q.append(9)
    elif item==10: 
       q.append(10)      
print(q)

1
1
9
3
8
6
2
8
8
6
3
1
1
3
6
1
1
6
1
3
3
5
1
9
6
3
7
6
10
7
2
1
2
1
6
6
9
3
3
3
9
5
6
1
8
6
1
3
4
9
10
9
3
2
8
6
6
1
9
2
1
8
6
8
9
8
5
6
2
3
4
10
1
5
10
3
3
6
1
9
6
6
10
3
6
9
3
5
6
6
3
9
3
5
1
3
5
3
3
3
8
4
1
8
1
6
1
3
3
5
3
8
6
3
8
6
6
9
9
6
2
9
3
3
2
6
6
6
2
8
2
9
9
1
9
8
3
9
6
1
9
3
3
2
6
8
6
6
1
7
6
6
3
1
1
9
7
10
5
9
2
3
6
6
2
1
10
3
1
6
6
1
9
9
9
6
6
9
1
4
1
9
9
1
8
10
1
3
8
1
6
9
6
1
9
9
3
1
9
8
6
10
1
10
6
9
2
1
3
1
6
10
10
6
6
9
6
10
3
3
1
10
3
5
10
8
3
6
9
2
5
1
8
8
6
3
3
3
9
3
3
2
9
5
3
2
10
3
5
6
2
3
6
3
3
6
9
6
2
10
5
1
10
3
9
1
7
3
6
6
10
6
9
8
10
6
6
10
10
3
4
2
4
3
1
3
6
6
1
8
1
4
10
7
3
8
6
6
1
2
10
6
7
8
1
5
6
9
2
4
10
1
1
6
7
2
1
9
1
2
5
6
10
10
7
1
5
2
6
4
3
1
3
6
3
9
9
3
1
2
6
2
1
5
1
9
7
1
1
3
2
6
10
6
7
2
9
6
3
5
2
1
6
5
6
3
2
1
6
9
3
10
1
2
1
9
3
1
1
3
9
8
9
8
2
10
6
8
6
6
6
3
1
1
1
6
3
6
2
7
1
2
2
1
3
3
3
6
9
7
6
6
10
8
3
9
2
2
7
5
4
2
3
6
1
6
9
5
6
3
1
2
9
7
9
9
2
6
6
6
5
10
6
10
5
1
2
3
1
7
2
5
9
1
2
6
7
9
7
9
6
6
1
6
1
2
10
3
6
10
9
7
2
3
6
6
10
2
8
6
1
1


In [0]:
df = pd.DataFrame(list(zip(ctrs, q)), 
               columns =['Id', 'label']) 
#df.to_csv('/content/drive/My Drive/submission22.csv',index=False)


In [0]:
df.to_csv('submission.csv',index=False)
!kaggle competitions submit fcis-sc-deeplearning-competition -f "submission.csv" -m "Submission from API"

100% 17.9k/17.9k [00:00<00:00, 87.2kB/s]
Successfully submitted to FCIS Deep Learning Competition